In [1]:
# Get API_KEY from .env
import os
from dotenv import load_dotenv
import openai
import json
import tiktoken
import pandas as pd
from collections import defaultdict
import numpy as np

In [2]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

config = {
    "API_KEY": API_KEY,
    "model": "gpt-3.5-turbo",
    "store": True,
}

In [ ]:
# Load dataset
df = pd.read_csv("./dataset/train.csv")

# Load OpenAI's token encoder for GPT-4 / GPT-3.5-turbo
tokenizer = tiktoken.get_encoding("cl100k_base")

# Function to estimate tokens for a given text
def estimate_tokens(text):
    return len(tokenizer.encode(text))

# Estimate tokens for each row in the dataset
df["token_count"] = df.apply(lambda row: estimate_tokens(row["question"] + " " + row["answer"]), axis=1)

# Total estimated tokens
total_tokens = df["token_count"].sum()

print(f"Total estimated tokens: {total_tokens}")


In [5]:
def convert_to_jsonl(dataframe, filename, is_test=False):
    jsonl_data = []
    for _, row in dataframe.iterrows():
        if not is_test:
            example = {
                "messages": [
                    {"role": "system", "content": "You are an AI grader that evaluates student answers on a scale from 0 to 5 in steps of 0.25."},
                    {"role": "user", "content": f"Question: {row['question']}\nAnswer: {row['answer']}"},
                    {"role": "assistant", "content": f"Score: {row['score']}"}
                ]
            }
        else:
            example = {
                "messages": [
                    {"role": "system", "content": "You are an AI grader that evaluates student answers on a scale from 0 to 5 in steps of 0.25."},
                    {"role": "user", "content": f"Question: {row['question']}\nAnswer: {row['answer']}"},
                    {"role": "assistant", "content": "Score:"}
                ],
                "id": row["ID"],
            }
        jsonl_data.append(example)

    # Save to JSONL file
    with open(filename, "w", encoding="utf-8") as f:
        for entry in jsonl_data:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [14]:
df = pd.read_csv("./dataset/train.csv")
test_df = pd.read_csv("./dataset/test.csv")
grouped_data = df.groupby("set")
grouped_data_test = test_df.groupby("set")
cou = 0
# for group, data in grouped_data:
#     print(f"Group: {group}")
#     unique_scores = data["score"].unique()
#     for score in unique_scores:
#         data_all = data[data["score"] == score]
#         print(f"Score: {score}")
#         print(f"Total: {len(data_all)}")
#         cou += len(data_all)
#     print()
# print(f"Total: {cou}")
col = df.columns
print(col)

num = {"Q1": 1, "Q2": 1, "Q3": 2, "Q4": 1}
# print(col)

path = "./dataset/processed/"
for group, data in grouped_data:
    # print(f"Group: {group}")
    unique_scores = data["score"].unique()

    # empty df for each group question and append each score later
    train_df, val_df = pd.DataFrame(columns=col), pd.DataFrame(columns=col)

    for score in unique_scores:
        data_all = data[data["score"] == score]
        # sample and drop for val set another is train set
        # print(f"Score: {score}")
        # print(f"Total: {len(data_all)}")
        num_sam = num[group]
        num_sam = max(0, min(num_sam, len(data_all)-1)) # reserved 1 for train
        # print(f"Sample: {num_sam}")
        val_data = data_all.sample(n=num_sam, random_state=42)
        train_data = data_all.drop(val_data.index)
        train_df = pd.concat([train_df, train_data])
        val_df = pd.concat([val_df, val_data])

    train_file = f"{path}train_{group}.jsonl"
    valid_file = f"{path}valid_{group}.jsonl"

    train_file_df = f"{path}train_{group}.csv"
    valid_file_df = f"{path}valid_{group}.csv"
    convert_to_jsonl(train_df, train_file)
    convert_to_jsonl(val_df, valid_file)
    # print(len(train_df), len(val_df))
    pd.DataFrame(train_df).to_csv(train_file_df, index=False)
    pd.DataFrame(val_df).to_csv(valid_file_df, index=False)

for group, data in grouped_data_test:
    test_file = f"{path}test_{group}.jsonl"
    test_file_df = f"{path}test_{group}.csv"
    convert_to_jsonl(data, test_file, is_test=True)

    pd.DataFrame(data).to_csv(test_file_df, index=False)
    # print(f"Group: {group}")
    # print(f"Total: {len(data)}")
    # print()


Index(['ID', 'set', 'question', 'answer', 'score'], dtype='object')


/tmp/ipykernel_14377/2764274778.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_df = pd.concat([train_df, train_data])
/tmp/ipykernel_14377/2764274778.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  val_df = pd.concat([val_df, val_data])
/tmp/ipykernel_14377/2764274778.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes.

In [9]:
types = ["train", "valid"]
# types = ["test"]
qs = ["Q1", "Q2", "Q3", "Q4"]
encoding = tiktoken.get_encoding("cl100k_base")

token_sum = 0

for t in types:
    for q in qs:
        data_path = f"{path}{t}_{q}.jsonl"
        print(f"Reading {data_path}")

        with open(data_path, 'r', encoding='utf-8') as f:
            dataset = [json.loads(line) for line in f]

        format_errors = defaultdict(int)

        for ex in dataset:
            if not isinstance(ex, dict):
                format_errors["data_type"] += 1
                continue
                
            messages = ex.get("messages", None)
            if not messages:
                format_errors["missing_messages_list"] += 1
                continue
                
            for message in messages:
                if "role" not in message or "content" not in message:
                    format_errors["message_missing_key"] += 1
                
                if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
                    format_errors["message_unrecognized_key"] += 1
                
                if message.get("role", None) not in ("system", "user", "assistant", "function"):
                    format_errors["unrecognized_role"] += 1
                    
                content = message.get("content", None)
                function_call = message.get("function_call", None)
                
                if (not content and not function_call) or not isinstance(content, str):
                    format_errors["missing_content"] += 1
            
            if not any(message.get("role", None) == "assistant" for message in messages):
                format_errors["example_missing_assistant_message"] += 1

        if format_errors:
            print("Found errors:")
            for k, v in format_errors.items():
                print(f"{k}: {v}")
        else:
            print("No errors found")
        
        n_missing_system = 0
        n_missing_user = 0
        n_messages = []
        convo_lens = []
        assistant_message_lens = []

        for ex in dataset:
            messages = ex["messages"]
            if not any(message["role"] == "system" for message in messages):
                n_missing_system += 1
            if not any(message["role"] == "user" for message in messages):
                n_missing_user += 1
            n_messages.append(len(messages))
            convo_lens.append(num_tokens_from_messages(messages))
            assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
            
        print("Num examples missing system message:", n_missing_system)
        print("Num examples missing user message:", n_missing_user)
        print_distribution(n_messages, "num_messages_per_example")
        print_distribution(convo_lens, "num_total_tokens_per_example")
        print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
        n_too_long = sum(l > 16385 for l in convo_lens)
        print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")
        token_sum += sum(convo_lens)
print(f"Total tokens: {token_sum}")

Reading ./dataset/processed/train_Q1.jsonl
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 435, 1408
mean / median: 619.170731707317, 557.0
p5 / p95: 474.7, 843.5000000000001

#### Distribution of num_assistant_tokens_per_example:
min / max: 6, 6
mean / median: 6.0, 6.0
p5 / p95: 6.0, 6.0

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning
Reading ./dataset/processed/train_Q2.jsonl
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 398, 1050
mean / median: 579.8915662650602, 567.0
p5 / p95: 448.6, 727.4

#### Distribution of num_as

No errors found
